In [2]:
pip install pyro-ppl

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.nn.functional as F # Import F for linear
import pyro
import pyro.distributions as dist
from pyro.nn import PyroSample, PyroModule
from torch.distributions import constraints
from pyro.infer import SVI, Trace_ELBO, Predictive
from pyro.optim import ClippedAdam
import numpy as np
import pandas as pd
import glob
import os
from sklearn.metrics import mean_squared_error
import math

In [4]:
# --- Configuration ---
# Set device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Data and Model Parameters
DATA_DIR = "/kaggle/input/training/Training Dataset" # Directory containing the CSV files
PREDICTION_FILE_INDEX = 20 # Index of the file to predict (e.g., 20 for IPF_Final_20.csv)
SEQ_LEN = 18 # Length of the input sequence
HIDDEN_DIM = 64
OUTPUT_DIM = 3
INPUT_DIM = 3 # Corresponds to the 3 columns used (cols 1, 2, 3)

# Training Parameters
NUM_STEPS = 2000
LEARNING_RATE = 1e-3
PATIENCE = 100 # For early stopping
MIN_DELTA = 1.0 # Minimum improvement for early stopping
NUM_PREDICTION_SAMPLES = 100 # Number of samples from posterior for prediction

Using device: cuda


In [5]:
# --- Data Loading ---

def get_file_paths(data_dir):
    """Gets and sorts file paths based on the index in the filename."""
    paths = glob.glob(os.path.join(data_dir, "IPF_Final_*.csv"))
    # Sort files numerically based on the index after 'IPF_Final_'
    paths.sort(key=lambda p: int(os.path.basename(p).split('_')[2].split('.')[0]))
    return paths

def load_training_data(paths, seq_len):
    """Loads sequence data for training."""
    print(f"Loading training data using sequence length: {seq_len}")
    # Need seq_len files for input and 1 file for the target
    num_files_needed = seq_len + 1
    if len(paths) < num_files_needed:
        raise ValueError(f"Need at least {num_files_needed} CSV files for training, but found {len(paths)}.")

    # Load the first seq_len+1 files
    # Ensure files exist before attempting to load
    for path in paths[:num_files_needed]:
        if not os.path.exists(path):
             raise FileNotFoundError(f"Required training file not found: {path}")

    arrays = [np.loadtxt(path, delimiter=',', skiprows=1, usecols=(1, 2, 3)) for path in paths[:num_files_needed]]

    # Stack the first seq_len arrays for the input sequence
    X_seq = np.stack(arrays[:seq_len], axis=0) # Shape: (seq_len, num_samples, input_dim)
    # The (seq_len+1)-th array is the target
    Y_arr = arrays[seq_len] # Shape: (num_samples, output_dim)

    # Transpose X_seq to (num_samples, seq_len, input_dim) as expected by LSTM batch_first=True
    x_tensor = torch.tensor(X_seq.transpose(1, 0, 2), dtype=torch.float32).to(device)
    y_tensor = torch.tensor(Y_arr, dtype=torch.float32).to(device)
    print(f"Training X shape: {x_tensor.shape}, Training Y shape: {y_tensor.shape}")
    return x_tensor, y_tensor

def load_prediction_data(paths, seq_len, prediction_file_index):
    """Loads the input sequence needed for prediction and the target file."""
    print(f"Loading data for predicting file index: {prediction_file_index}")
    # Indices are 1-based in filenames, list indices are 0-based
    target_file_name = f"IPF_Final_{prediction_file_index}.csv"
    target_file_path = os.path.join(DATA_DIR, target_file_name)

    if target_file_path not in paths:
          raise FileNotFoundError(f"Target file {target_file_path} not found in the dataset.")

    # Find the index of the target file in the sorted list
    target_list_index = paths.index(target_file_path)

    # The input sequence consists of seq_len files *before* the target file
    start_index = target_list_index - seq_len
    end_index = target_list_index # Exclusive

    if start_index < 0:
        # Find the first available index to make the sequence start from there
        actual_seq_len = target_list_index
        if actual_seq_len == 0:
             raise ValueError(f"Prediction file {prediction_file_index} is the first file. Cannot form a sequence of length {seq_len}.")
        print(f"Warning: Not enough preceding files to form a sequence of length {seq_len} for prediction file {prediction_file_index}.")
        print(f"Using a shorter sequence of length {actual_seq_len} starting from file index {paths[0].split('_')[-1].split('.')[0]}.")
        start_index = 0 # Start from the first available file
        # Note: This will use a shorter sequence than SEQ_LEN if start_index < 0 originally.
        # The LSTM handles variable length sequences, but training is done with fixed SEQ_LEN.
        # This might lead to a train/predict mismatch if sequence length is critical.
        # For robustness, you might want to require sufficient preceding files for prediction too.
        # Let's keep the original error behavior to maintain consistency with expected input sequence length.
        raise ValueError(f"Not enough preceding files ({target_list_index}) to form a sequence of length {seq_len} for prediction file {prediction_file_index}.")


    input_paths = paths[start_index:end_index]
    print(f"Using files {os.path.basename(input_paths[0])} to {os.path.basename(input_paths[-1])} for prediction input.")

    # Ensure input files exist
    for path in input_paths:
         if not os.path.exists(path):
             raise FileNotFoundError(f"Required prediction input file not found: {path}")

    # Load the sequence data
    input_arrays = [np.loadtxt(path, delimiter=',', skiprows=1, usecols=(1, 2, 3)) for path in input_paths]
    X_pred_seq = np.stack(input_arrays, axis=0) # Shape: (seq_len, num_samples, input_dim)

    # Load the target data
    Y_pred_arr = np.loadtxt(target_file_path, delimiter=',', skiprows=1, usecols=(1, 2, 3)) # Shape: (num_samples, output_dim)

    # Transpose X_pred_seq for LSTM input
    x_pred_tensor = torch.tensor(X_pred_seq.transpose(1, 0, 2), dtype=torch.float32).to(device)
    y_pred_tensor = torch.tensor(Y_pred_arr, dtype=torch.float32).to(device)
    print(f"Prediction X shape: {x_pred_tensor.shape}, Prediction Y shape: {y_pred_tensor.shape}")
    return x_pred_tensor, y_pred_tensor

In [6]:
def load_prediction_data(paths, seq_len, prediction_file_index):
    """Loads the input sequence needed for prediction and the target file."""
    print(f"Loading data for predicting file index: {prediction_file_index}")
    # Indices are 1-based in filenames, list indices are 0-based
    target_file_path = os.path.join(DATA_DIR, f"IPF_Final_{prediction_file_index}.csv")

    if target_file_path not in paths:
         raise FileNotFoundError(f"Target file {target_file_path} not found in the dataset.")

    # Find the index of the target file in the sorted list
    target_list_index = paths.index(target_file_path)

    # The input sequence consists of seq_len files *before* the target file
    start_index = target_list_index - seq_len
    end_index = target_list_index # Exclusive

    if start_index < 0:
        raise ValueError(f"Not enough preceding files ({start_index}) to form a sequence of length {seq_len} for prediction file {prediction_file_index}.")

    input_paths = paths[start_index:end_index]
    print(f"Using files {os.path.basename(input_paths[0])} to {os.path.basename(input_paths[-1])} for prediction input.")

    # Load the sequence data
    input_arrays = [np.loadtxt(path, delimiter=',', skiprows=1, usecols=(1, 2, 3)) for path in input_paths]
    X_pred_seq = np.stack(input_arrays, axis=0) # Shape: (seq_len, num_samples, input_dim)

    # Load the target data
    Y_pred_arr = np.loadtxt(target_file_path, delimiter=',', skiprows=1, usecols=(1, 2, 3)) # Shape: (num_samples, output_dim)

    # Transpose X_pred_seq for LSTM input
    x_pred_tensor = torch.tensor(X_pred_seq.transpose(1, 0, 2), dtype=torch.float32).to(device)
    y_pred_tensor = torch.tensor(Y_pred_arr, dtype=torch.float32).to(device)
    print(f"Prediction X shape: {x_pred_tensor.shape}, Prediction Y shape: {y_pred_tensor.shape}")
    return x_pred_tensor, y_pred_tensor

In [22]:
# --- Model Definition ---

class MultiLSTMBackbone(PyroModule):
    """
    A Bayesian LSTM backbone using multiple LSTMs and combining their outputs.
    Weights and biases of LSTMs and FC layers are treated as random variables.
    Uses PyroSample for weights/biases and F.linear for the linear transformation.
    """
    def __init__(self, input_size=3, hidden_size=64):
        super().__init__()
        self.hidden_size = hidden_size
        self.input_size = input_size
        # Calculate input/output sizes for the linear layers
        hidden_fc_input_size = hidden_size * 3
        fc_output_size = 1 # Each linear layer outputs 1 dimension

        # Define Bayesian LSTM layers using PyroModule
        self.lstm_phi1 = PyroModule[nn.LSTM](input_size, hidden_size, batch_first=True)
        # The default PyroSample priors for nn.LSTM should be fine here
        # (handled automatically by PyroModule[nn.LSTM])

        self.lstm_phi = PyroModule[nn.LSTM](input_size, hidden_size, batch_first=True)
         # The default PyroSample priors for nn.LSTM should be fine here

        self.lstm_phi2 = PyroModule[nn.LSTM](input_size, hidden_size, batch_first=True)
         # The default PyroSample priors for nn.LSTM should be fine here

        # Define Bayesian FC layer parameters explicitly using PyroSample
        # We will use F.linear in the forward pass with these sampled parameters.
        # This avoids potential issues with PyroModule[nn.Linear]'s internal handling.
        self.fc1_weight = PyroSample(lambda self: dist.Normal(
            torch.zeros(fc_output_size, hidden_fc_input_size, device=device),
            torch.ones(fc_output_size, hidden_fc_input_size, device=device)
        ).to_event(2)) # Sample shape (1, 192), treat as single variable

        self.fc1_bias = PyroSample(lambda self: dist.Normal(
            torch.zeros(fc_output_size, device=device),
            torch.ones(fc_output_size, device=device)
        ).to_event(1)) # Sample shape (1,), treat as single variable

        self.fc2_weight = PyroSample(lambda self: dist.Normal(
            torch.zeros(fc_output_size, hidden_fc_input_size, device=device),
            torch.ones(fc_output_size, hidden_fc_input_size, device=device)
        ).to_event(2)) # Sample shape (1, 192), treat as single variable

        self.fc2_bias = PyroSample(lambda self: dist.Normal(
            torch.zeros(fc_output_size, device=device),
            torch.ones(fc_output_size, device=device)
        ).to_event(1)) # Sample shape (1,), treat as single variable

        self.fc3_weight = PyroSample(lambda self: dist.Normal(
            torch.zeros(fc_output_size, hidden_fc_input_size, device=device),
            torch.ones(fc_output_size, hidden_fc_input_size, device=device)
        ).to_event(2)) # Sample shape (1, 192), treat as single variable

        self.fc3_bias = PyroSample(lambda self: dist.Normal(
            torch.zeros(fc_output_size, device=device),
            torch.ones(fc_output_size, device=device)
        ).to_event(1)) # Sample shape (1,), treat as single variable


    def forward(self, x):
        """Forward pass through the multi-LSTM backbone."""
        # Ensure input is on the same device as model parameters
        # Use a parameter's device that is guaranteed to exist (e.g., one of the sampled weights)
        try:
            target_device = self.fc1_weight.device
        except AttributeError:
             if pyro.settings.get("enable_validation", False):
                  print("Warning: Could not access self.fc1_weight.device. This might happen during early tracing or if samples are not yet available. Falling back to global device.")
             target_device = device

        x = x.to(target_device)


        # Pass input through each LSTM
        # Note: If you wanted Bayesian LSTMs *without* PyroModule[nn.LSTM],
        # you would sample weight/bias matrices here and use F.lstm, similar to F.linear below.
        # PyroModule[nn.LSTM] handles sampling its parameters internally in the model.
        _, (h1, _) = self.lstm_phi1(x) # h1 shape: (1, batch_size, hidden_size)
        _, (h2, _) = self.lstm_phi(x)
        _, (h3, _) = self.lstm_phi2(x)

        # Concatenate the last hidden states from each LSTM
        # h_n[-1] accesses the hidden state of the last layer
        h_cat = torch.cat([h1[-1], h2[-1], h3[-1]], dim=1) # Shape: (batch_size, hidden_size * 3)

        # Pass concatenated hidden states through the linear layers using sampled parameters
        # F.linear handles the batch dimension correctly.
        out1 = F.linear(h_cat, self.fc1_weight, self.fc1_bias) # Shape: (batch_size, 1)
        out2 = F.linear(h_cat, self.fc2_weight, self.fc2_bias) # Shape: (batch_size, 1)
        out3 = F.linear(h_cat, self.fc3_weight, self.fc3_bias) # Shape: (batch_size, 1)

        # Concatenate the outputs of the FC layers
        final_out = torch.cat([out1, out2, out3], dim=1) # Shape: (batch_size, 3)
        return final_out


In [23]:
class BayesianLSTMModel(PyroModule):
    """
    The complete Bayesian LSTM model including the backbone and observation likelihood.
    """
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        # Instantiate the backbone network
        self.backbone = MultiLSTMBackbone(input_dim, hidden_dim)

    def forward(self, x, y=None):
        """
        Generative model p(y, z | x) where z are the latent parameters.
        Defines the generative process:
        1. Samples the parameters z (implicitly via PyroSample in backbone).
        2. Runs the input `x` through the backbone to get mean predictions.
        3. Samples observation noise scale `obs_scale`.
        4. Samples the output `y` from a Normal distribution centered at the backbone's output.
        """
        # Get the mean prediction from the backbone
        # This call implicitly samples parameters defined with PyroSample in the backbone
        out_mean = self.backbone(x) # Shape: (batch_size, output_dim)

        # Ensure output mean is on the correct device before sampling obs_scale
        out_mean = out_mean.to(device)

        # Sample observation noise standard deviation (sigma)
        # Sample one observation noise standard deviation *per output dimension*, shared across batch.
        # .to_event(1) ensures the sample site 'obs_scale' is treated as a single random variable (a vector of scales)
        # Sampled OUTSIDE the data plate.
        obs_scale = pyro.sample("obs_scale",
                                dist.Uniform(0.5, 2.0).expand([self.output_dim]).to_event(1)
                               ).to(out_mean.device) # Ensure scale is on same device

        # Plate over the batch dimension for the observation likelihood
        with pyro.plate("data", size=x.size(0)):
            # Expand scale to match the output shape for the Normal distribution
            # Since obs_scale is shape (output_dim,), expanding it to out_mean.shape (batch_size, output_dim)
            # will tile it across the batch dimension, which is correct for shared scales.
            scale_expanded = obs_scale.expand(out_mean.shape)

            # Sample the observed data `y` using a Normal likelihood
            # The `to_event(1)` treats the output dimensions for a single data point as conditionally independent
            # given the mean (from the backbone) and the scale vector.
            pyro.sample("obs",
                        dist.Normal(out_mean, scale_expanded).to_event(1),
                        obs=y) # Pass observed data `y` if training/evaluating likelihood

        return out_mean # Return the mean prediction (useful for prediction)


In [25]:
# --- Main Execution ---

if __name__ == "__main__":

    # --- Load Data ---
    all_paths = get_file_paths(DATA_DIR)
    # Check if enough files exist overall
    # Need SEQ_LEN + 1 for training OR PREDICTION_FILE_INDEX for prediction
    # The path list is sorted, so the highest index needed is the prediction index file.
    # If PREDICTION_FILE_INDEX is less than SEQ_LEN+1, we still need SEQ_LEN+1 files for training.
    # The prediction needs PREDICTION_FILE_INDEX and the SEQ_LEN files preceding it, meaning files from
    # index (PREDICTION_FILE_INDEX - SEQ_LEN) up to PREDICTION_FILE_INDEX (inclusive of target).
    # So the highest index file needed is PREDICTION_FILE_INDEX.
    # The training needs files from index 1 up to SEQ_LEN+1.
    # We need files up to max(SEQ_LEN + 1, PREDICTION_FILE_INDEX).
    required_max_index = max(SEQ_LEN + 1, PREDICTION_FILE_INDEX)
    # Check if the path list contains the file with index required_max_index
    required_file_path = os.path.join(DATA_DIR, f"IPF_Final_{required_max_index}.csv")

    if required_file_path not in all_paths:
         last_file_index = int(os.path.basename(all_paths[-1]).split('_')[2].split('.')[0]) if all_paths else 0
         raise FileNotFoundError(f"Dataset directory '{DATA_DIR}' does not contain enough files. Needed file 'IPF_Final_{required_max_index}.csv' for training or prediction setup. Found files up to index {last_file_index}.")


    x_train_tensor, y_train_tensor = load_training_data(all_paths, SEQ_LEN)
    x_pred_tensor, y_pred_tensor = load_prediction_data(all_paths, SEQ_LEN, PREDICTION_FILE_INDEX)

    # --- Initialize Model and Guide ---
    model = BayesianLSTMModel(INPUT_DIM, HIDDEN_DIM, OUTPUT_DIM).to(device)
    # AutoNormal guide approximates the posterior distribution of parameters with Gaussians
    guide = pyro.infer.autoguide.AutoNormal(model)

    # --- Training Setup ---
    pyro.clear_param_store() # Clear parameter store before training
    # ClippedAdam optimizer helps prevent exploding gradients
    optimizer = ClippedAdam({"lr": LEARNING_RATE})
    # Stochastic Variational Inference (SVI) with Trace Evidence Lower Bound (ELBO) loss
    svi = SVI(model, guide, optimizer, loss=Trace_ELBO())

    # --- Training Loop ---
    print("\n--- Starting Training ---")
    best_loss = float('inf')
    no_improve_count = 0

    for step in range(NUM_STEPS):
        # Perform one optimization step
        loss = svi.step(x_train_tensor, y_train_tensor)

        if step % 50 == 0 or step == NUM_STEPS - 1:
            print(f"Step {step:>4} - ELBO Loss: {loss:.4f}")

        # Early stopping check
        # Use a small tolerance for float comparison
        if loss < best_loss - MIN_DELTA:
            best_loss = loss
            no_improve_count = 0
            # Optionally: save model parameters here if loss improves
            # pyro.get_param_store().save("best_model.params")
        else:
            no_improve_count += 1

        if no_improve_count >= PATIENCE:
            print(f"\n⏹️ Early stopping triggered at step {step} due to no improvement for {PATIENCE} steps.")
            break

    print("--- Training Finished ---")

    # --- Prediction ---
    print(f"\n--- Generating Predictions for file index {PREDICTION_FILE_INDEX} ---")
    # Use Predictive to sample from the posterior predictive distribution
    # It runs the model forward multiple times, sampling from the guide each time
    predictive = Predictive(model, guide=guide, num_samples=NUM_PREDICTION_SAMPLES)

    # Get predictions for the prediction input tensor
    # No need for y_pred_tensor here, as we are generating predictions
    # The model's forward pass handles the device placement of x_pred_tensor
    posterior_samples = predictive(x_pred_tensor)

    # 'obs' contains samples from the likelihood N(backbone(x), obs_scale)
    # Shape: (num_samples, batch_size, output_dim)
    y_pred_samples = posterior_samples['obs'].detach().cpu().numpy()

    # Calculate the mean prediction across samples
    # Shape: (batch_size, output_dim)
    y_pred_mean = y_pred_samples.mean(axis=0)

    # Calculate the standard deviation across samples (uncertainty estimate)
    y_pred_std = y_pred_samples.std(axis=0)

    print(f"Generated {NUM_PREDICTION_SAMPLES} posterior samples.")
    print(f"Mean prediction shape: {y_pred_mean.shape}")
    print(f"Prediction uncertainty (std dev) shape: {y_pred_std.shape}")


    # --- Evaluation ---
    print("\n--- Evaluating Predictions ---")
    # Get the actual target values
    y_actual = y_pred_tensor.cpu().numpy() # Shape: (batch_size, output_dim)

    # Calculate RMSE for each output dimension
    rmse_dim = []
    for i in range(OUTPUT_DIM):
        mse_dim_i = mean_squared_error(y_actual[:, i], y_pred_mean[:, i])
        rmse_dim_i = math.sqrt(mse_dim_i)
        rmse_dim.append(rmse_dim_i)
        print(f"RMSE for Output Dimension {i+1}: {rmse_dim_i:.4f}")

    # Calculate overall RMSE across all dimensions
    overall_mse = mean_squared_error(y_actual.flatten(), y_pred_mean.flatten())
    overall_rmse = math.sqrt(overall_mse)
    print(f"Overall RMSE: {overall_rmse:.4f}")

    # Example: Print first 5 predictions vs actuals
    print("\n--- Sample Predictions vs Actuals (First 5) ---")
    for i in range(min(5, y_actual.shape[0])):
        print(f"Sample {i+1}:")
        print(f"  Actual: {np.round(y_actual[i], 3)}")
        print(f"  Predicted Mean: {np.round(y_pred_mean[i], 3)}")
        print(f"  Predicted StdDev: {np.round(y_pred_std[i], 3)}")

    print("\n--- Script Finished ---")

Using device: cuda
Loading training data using sequence length: 18
Training X shape: torch.Size([10004, 18, 3]), Training Y shape: torch.Size([10004, 3])
Loading data for predicting file index: 20
Using files IPF_Final_1.csv to IPF_Final_19.csv for prediction input.
Prediction X shape: torch.Size([10004, 18, 3]), Prediction Y shape: torch.Size([10004, 3])

--- Starting Training ---
Step    0 - ELBO Loss: 88070484.7034
Step   50 - ELBO Loss: 83271817.7628
Step  100 - ELBO Loss: 69819734.4022
Step  150 - ELBO Loss: 52214511.1879
Step  200 - ELBO Loss: 42453460.9271
Step  250 - ELBO Loss: 38147586.9874
Step  300 - ELBO Loss: 30634857.6202
Step  350 - ELBO Loss: 24832190.7231
Step  400 - ELBO Loss: 23197650.4706
Step  450 - ELBO Loss: 21951287.2434
Step  500 - ELBO Loss: 19166266.9342
Step  550 - ELBO Loss: 18170914.3102
Step  600 - ELBO Loss: 15846260.4510
Step  650 - ELBO Loss: 15112913.6985
Step  700 - ELBO Loss: 14359687.0287
Step  750 - ELBO Loss: 13560104.0832
Step  800 - ELBO Loss: 

In [27]:
# --- Save Predictions to CSV ---
print(f"\n--- Saving Predictions to CSV ---")

# Define column names for the CSV
# Separate columns for the mean and standard deviation of each output dimension
mean_cols = [f'pred_mean_dim{i+1}' for i in range(OUTPUT_DIM)]
std_cols = [f'pred_std_dim{i+1}' for i in range(OUTPUT_DIM)]
column_names = mean_cols + std_cols

# Combine the mean predictions and standard deviations side-by-side
# The shape will be (batch_size, output_dim * 2)
predicted_data_with_uncertainty = np.hstack((y_pred_mean, y_pred_std))

# Create a pandas DataFrame from the combined data
df_predictions = pd.DataFrame(predicted_data_with_uncertainty, columns=column_names)

# Define the output filename based on the prediction file index
output_filename = f"predictions_IPF_Final_{PREDICTION_FILE_INDEX}.csv"

# Save the DataFrame to a CSV file
# index=False prevents pandas from writing the DataFrame index as a column
df_predictions.to_csv(output_filename, index=False)

print(f"Predictions (mean and std dev) saved to {output_filename}")
print("\n--- Script Finished ---")


--- Saving Predictions to CSV ---
Predictions (mean and std dev) saved to predictions_IPF_Final_20.csv

--- Script Finished ---
